In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import make_interp_spline

In [ ]:
url = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vRBUgfYBUq8heoHl7sFbhfvRTPM58qTyzsEEJkeILT11GIFgqSt4piO_F4rvWs8w-eZjjQvXw3-2Hzz/pub?gid=1854992701&single=true&output=csv')
df = pd.DataFrame(url).drop(['Unnamed: 0'],axis=1).dropna()

In [ ]:
Sales = ['NA_Sales','EU_Sales','JP_Sales','Other_Sales','Total_Sales']

In [ ]:
def to_int(x):
    return int(x)

def modify(x):
    if x < 50:
      x = x + 2000
    elif x < 100:
      x = x + 1900
    return x

df['Year'] = df['Year'].apply(to_int).apply(modify)

def to_float(x):
    x = x.replace('K','000')
    x = x.replace('M','000000')
    return float(x)

for Column_Name in Sales[:-1]:
    df[Column_Name] = df[Column_Name].apply(to_float)

df['Total_Sales'] = df['NA_Sales'] + df['EU_Sales'] + df['JP_Sales'] + df['Other_Sales']

df = df.sort_values(by=['Year'], ascending=[False]).reset_index()
df = df.drop(['index'], axis = 1)

df

In [ ]:
Genre = []
for x in df['Genre']:
  if x not in Genre:
    Genre.append(x)
Genre.remove('Misc') # 범례를 알 수 없는경우는 뺀다

In [ ]:
# Year, Genre, Sales 만 뽑아내기
df_ygs = pd.DataFrame(columns = ['Year','Genre']+Sales)

Year_S = 2020
Year_E = 1980

Y = Year_S
n = 0
while Y >= Year_E:
  for N1 in Genre:
    df_ygs.loc[n,'Year'] = Y
    df_ygs.loc[n,'Genre'] = N1
    n = n+1
  Y = Y - 1

Y = Year_S
while Y >= Year_E:
  for N in Genre:
    for S in Sales:
      Sum = df[df['Year']==Y][df['Genre']==N][S].sum()
      Index = df_ygs[df_ygs['Year']==Y][df_ygs['Genre']==N].index[0]
      df_ygs.loc[Index,S] = Sum
  Y = Y - 1

df_ygs

In [ ]:
# 결과물을 각 년도별 비율로 환산

df_ygs_ratio = df_ygs.copy()

Year_S = 2020
Year_E = 1980

Y = Year_S
while Y >= Year_E:
  for Area in Sales:
    Sum = df_ygs_ratio[df_ygs_ratio['Year'] == Y][Area].sum()
    List = df_ygs_ratio[df_ygs_ratio['Year'] == Y].index
    for L in List:
      df_ygs_ratio.loc[L,Area] = 100*df_ygs_ratio.loc[L,Area]/Sum
  Y = Y - 1

In [ ]:
for S in Sales:
  plt.figure(figsize=(28,5))
  for N in Genre:
    x = (df_ygs_ratio[df_ygs_ratio['Genre'] == N]['Year'])
    y = (df_ygs_ratio[df_ygs_ratio['Genre'] == N][S])
    plt.plot(x,y,label=N)
    plt.legend(loc=2 ,fontsize=14);
    plt.title(S)

plt.show()

In [ ]:
# +- 1년씩

df_ygs_copy = df_ygs.copy()
df_ygs_mod = pd.DataFrame(columns = ['Year','Genre']+Sales)

Year_S = 2020
Year_E = 1980

Y = Year_S
n = 0
while Y >= Year_E:
  for N1 in Genre:
    df_ygs_mod.loc[n,'Year'] = Y
    df_ygs_mod.loc[n,'Genre'] = N1
    for i in Sales:
      df_ygs_mod.loc[n,i] = 0
    n = n+1
  Y = Y - 1

Y = Year_S
n = 0
while Y >= Year_E:
  for Area in Sales:
    for N in Genre:
      List_index = df_ygs_copy[df_ygs['Genre'] == N].index
      n = 1
      while n < len(List_index)-1:
        df_ygs_mod.loc[List_index[n-1],Area] = df_ygs_mod.loc[List_index[n-1],Area] + df_ygs_copy.loc[List_index[n],Area]/3
        df_ygs_mod.loc[List_index[n],Area] = df_ygs_mod.loc[List_index[n],Area] + df_ygs_copy.loc[List_index[n],Area]/3
        df_ygs_mod.loc[List_index[n+1],Area] = df_ygs_mod.loc[List_index[n+1],Area] + df_ygs_copy.loc[List_index[n],Area]/3
        n = n + 1
  Y = Y - 1

In [ ]:
# 한번 더

df_ygs_mod_copy = df_ygs_mod.copy()
df_ygs_mod2 = pd.DataFrame(columns = ['Year','Genre']+Sales)

Year_S = 2020
Year_E = 1980

Y = Year_S
n = 0
while Y >= Year_E:
  for N1 in Genre:
    df_ygs_mod2.loc[n,'Year'] = Y
    df_ygs_mod2.loc[n,'Genre'] = N1
    for i in Sales:
      df_ygs_mod2.loc[n,i] = 0
    n = n+1
  Y = Y - 1

Y = Year_S
n = 0
while Y >= Year_E:
  for Area in Sales:
    for N in Genre:
      List_index = df_ygs_mod_copy[df_ygs['Genre'] == N].index
      n = 1
      while n < len(List_index)-1:
        df_ygs_mod2.loc[List_index[n-1],Area] = df_ygs_mod2.loc[List_index[n-1],Area] + df_ygs_mod_copy.loc[List_index[n],Area]/3
        df_ygs_mod2.loc[List_index[n],Area] = df_ygs_mod2.loc[List_index[n],Area] + df_ygs_mod_copy.loc[List_index[n],Area]/3
        df_ygs_mod2.loc[List_index[n+1],Area] = df_ygs_mod2.loc[List_index[n+1],Area] + df_ygs_mod_copy.loc[List_index[n],Area]/3
        n = n + 1
  Y = Y - 1

In [ ]:
# 결과물을 각 년도별 비율로 환산

df_ygs_mod2_ratio = df_ygs_mod2.copy()

Year_S = 2020
Year_E = 1980

Y = Year_S
while Y >= Year_E:
  for Area in Sales:
    Sum = df_ygs_mod2_ratio[df_ygs_mod2_ratio['Year'] == Y][Area].sum()
    List = df_ygs_mod2_ratio[df_ygs_mod2_ratio['Year'] == Y].index
    for L in List:
      df_ygs_mod2_ratio.loc[L,Area] = 100*df_ygs_mod2_ratio.loc[L,Area]/Sum
  Y = Y - 1

In [ ]:
for S in Sales:
  plt.figure(figsize=(28,5))
  for N in Genre:
    x = (df_ygs_mod2_ratio[df_ygs['Genre'] == N]['Year'])
    y = (df_ygs_mod2_ratio[df_ygs['Genre'] == N][S])
    plt.plot(x,y,label=N)
    plt.legend(loc=2 ,fontsize=14);
    plt.title(S)

plt.show()

In [ ]:
def vs_Sales(DataFrame,Column_Name):
  List = []
  for x in DataFrame[Column_Name]:
    if x not in List:
      List.append(x)
  extract = pd.DataFrame(columns = [Column_Name]+Sales)
  n = 0
  while n < len(List) - 1:
    dummy = DataFrame[DataFrame[Column_Name] == List[n]]
    extract.loc[n,Column_Name] = List[n]
    for x in Sales:
      extract.loc[n,x] = dummy[x].sum()
    n = n+1
  return extract

In [ ]:
dfy = vs_Sales(df,'Year')

In [ ]:
plt.figure(figsize=(28,5))

for N in Sales:
  x = (dfy['Year'])
  y = (dfy[N])
  plt.plot(x,y,label=N)

plt.legend(loc=2 ,fontsize=14);
plt.title('Local Rate')

plt.show()

In [ ]:
df[df['NA_Sales']].sort_values(by=['NA_Sales'], ascending=[False])

#df = df.sort_values(by=['Year'], ascending=[False]).reset_index()
#df = df.drop(['index'], axis = 1)

In [ ]:
df_ygs[df_ygs['Year'] == 1997]

In [ ]:
df2 = df.copy()

df2['Current_Year_Sales'] = df2['Total_Sales']

n = 2020
while n >= 1980:
  df2['Current_Year_Sales'][df2['Year'] == n] = df2[df['Year'] == n]['Total_Sales'].sum()
  n = n - 1

dfdf = df2.copy()

for i in Sales:
  dfdf[i] = 100*dfdf[i]/dfdf['Current_Year_Sales']

dfdf[dfdf['JP_Sales'] > 50]

In [ ]:
dfdf = df.copy()

List = []
for S in Sales:
  var = (dfdf[S]-dfdf[S].mean())**2/len(dfdf)
  std = var / np.std(dfdf[S])
  for x in std[std > 500].index:
    List.append(x)

In [ ]:
df3 = df2.loc[List].sort_values(by=['Year'], ascending=[False]).copy()
df3 = df3.loc[~df3.index.duplicated(keep='first')]

df3

In [ ]:
df3['Total_Sales'].sum()/df['Total_Sales'].sum()

In [ ]:
len(df3)

In [ ]:
def pie_graph(DataFrame,Label):
  total_share = 100 * DataFrame[Sales].sum() / DataFrame[Sales].sum()[len(Sales)-1]
  plt.figure(figsize=(32,5)) # 캔버스 크기 선택
  n = 0
  while n < len(Sales):
    Area = Sales[n]
    dummy = DataFrame.sort_values(by=[Area], ascending=[False]).copy() # Area에 따라 내림차순

    fig = plt.subplot(1,len(Sales),n+1) # 1행 len(Sales)열의 n+1 번째 캔버스가 fig
    fig.pie(x=dummy[Area],labels=dummy[Label],autopct=lambda p : '{:.2f}%'.format(p)) # fig에 파이 차트 저장? 입력?
    plt.title(Area+"("+str(total_share[n].round(1))+"%)") # fig의 title은 Area 변수

    n = n+1

  plt.show()

In [ ]:
pie_graph(df3,'Genre')

In [ ]:
GS = vs_Sales(df,'Genre')

pie_graph(GS,'Genre')

In [ ]:
dfp = vs_Sales(df,'Platform')

In [ ]:
pie_graph(dfp,'Platform')

In [ ]:
df_Role = df[df['Genre'] == 'Role-Playing'].copy()

In [ ]:
df_Role_p = vs_Sales(df_Role,'Platform')

In [ ]:
pie_graph(df_Role_p,'Platform')

In [ ]:
Platform = []
for x in df['Platform']:
  if x not in Platform:
    Platform.append(x)

In [ ]:
# Year, Platform, Sales 만 뽑아내기(Genre == 'Role-Plyind)
df_yps = pd.DataFrame(columns = ['Year','Platform']+Sales)
df_yps_R = pd.DataFrame(columns = ['Year','Platform']+Sales)
df_R = df[df['Genre'] == 'Role-Playing'].copy()

Year_S = 2020
Year_E = 1980

Y = Year_S
n = 0
while Y >= Year_E:
  for N in Platform:
    df_yps.loc[n,'Year'] = Y
    df_yps.loc[n,'Platform'] = N
    n = n+1
  Y = Y - 1

Y = Year_S
while Y >= Year_E:
  for N in Platform:
    for S in Sales:
      Sum = df[df['Year']==Y][df['Platform']==N][S].sum()
      Index = df_yps[df_yps['Year']==Y][df_yps['Platform']==N].index[0]
      df_yps.loc[Index,S] = Sum
  Y = Y - 1

Y = Year_S
n = 0
while Y >= Year_E:
  for N in Platform:
    df_yps_R.loc[n,'Year'] = Y
    df_yps_R.loc[n,'Platform'] = N
    n = n+1
  Y = Y - 1

Y = Year_S
while Y >= Year_E:
  for N in Platform:
    for S in Sales:
      Sum = df_R[df_R['Year']==Y][df_R['Platform']==N][S].sum()
      Index = df_yps_R[df_yps_R['Year']==Y][df_yps_R['Platform']==N].index[0]
      df_yps_R.loc[Index,S] = Sum
  Y = Y - 1

In [ ]:
df_yps

In [ ]:
df_yps_R

In [ ]:
  S = 'Other_Sales'
  plt.figure(figsize=(28,5))
  for N in Platform:
    x = (df_yps[df_yps['Platform'] == N]['Year'])
    y = (df_yps[df_yps['Platform'] == N][S])
    plt.plot(x,y,label=N)
    plt.legend(loc='upper left', ncol=5 ,fontsize=14);
    plt.title(S)

  plt.figure(figsize=(28,5))  
  for N in Platform:
    x = (df_yps_R[df_yps_R['Platform'] == N]['Year'])
    y = (df_yps_R[df_yps_R['Platform'] == N][S])
    plt.plot(x,y,label=N)
    plt.legend(loc='upper left', ncol=5 ,fontsize=14);
    plt.title(S+'(Only Role-Playing)')

  plt.show()

In [ ]:
df[df['Name'] == 'Smarty Pants']

In [ ]:
  S = 'Other_Sales'
  plt.figure(figsize=(28,5))
  for N in Platform:
    x = (df_yps_R[df_yps_R['Platform'] == N]['Year'])
    y = (df_yps_R[df_yps_R['Platform'] == N][S])
    plt.plot(x,y,label=N)
    plt.legend(loc='upper left', ncol=5 ,fontsize=14);
    plt.title(S)

  plt.show()